# 1. Environment setup

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from scipy import stats

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from utility_dot_py.utility import QueryProcessor
from assistments_workbench.data_access.db_access.connector import db
from assistments_workbench.data_access.attach_query_result import attach_query_result_by_header
from assistments_workbench.config_reader import config
from assistments_workbench.data_converters.PFAConverter import PFAConverter
from assistments_workbench.models.model import classification_model

In [ ]:
# make images show up and large for presentation
%matplotlib inline
pylab.rcParams['figure.figsize'] = 9, 6
pylab.rcParams['font.size'] = 14

In [ ]:
# define output folder
output_dir = config.get('localfiles', 'data_path')

# 2. Run a SQL query and save results to a local file

This is the SQL query for getting user id, problem id, problem set id, and correctness from ASSISTments database

In [ ]:

sql_query = """ select user_id, sequence_id, problem_id, correct from problem_logs pl
left join class_assignments ca on ca.id = pl.assignment_id
where ca.sequence_id in (
5968,11898,6921,5969,37570
) and pl.start_time >= '2015-01-01' and pl.start_time < '2015-12-01' and pl.correct is not null
and pl.original = 1
order by user_id, pl.id; """

Initialize and run the query processor: 

In [ ]:
processor = QueryProcessor(db, output_dir)

In [ ]:
sql_data_path = processor.run_query_to_csv(sql_query)

In [ ]:
print sql_data_path

# 3. Attach additional features to the data set

Based on what we already have, automatically attach additional features to the data set, for exmaple, add new _problem difficutly_ information to the data set becuase we have _problem id_

In [ ]:
sql_with_features = attach_query_result_by_header(sql_data_path)

In [ ]:
print sql_with_features

# 4. Convert the data set to the PFA feature set

In [ ]:
col_mapping = {'user_id': 0, 'sequence_id' : 1, 'problem_id' : 2, 'correct': 3, 'difficulty': 4}

In [ ]:
pfa_data_path = PFAConverter(sql_with_features, None, col_mapping, 1)

In [ ]:
print pfa_data_path

# 5. Data exploration

In [ ]:
input_data = pd.read_csv(pfa_data_path)

Show first few row of the data set:

In [ ]:
input_data.head()

Show basic data statistics:

In [ ]:
input_data.describe()

In [ ]:
# mean of probelm difficulty
np.mean(input_data['difficulty'])

# 6. Data visualization

In [ ]:
input_data.skill_id.value_counts().plot(kind='barh')
plt.title('Item counts by skill id')
plt.xlabel('Item counts')
plt.ylabel('Skill id')

In [ ]:
# hisogtam of difficulty
input_data['difficulty'].hist()
plt.title('Histogram of Difficulty')
plt.xlabel('Difficulty')
plt.ylabel('Frequency')

# 7. Feature engineering
Add z-socre of difficulty as a new feature:

In [ ]:
input_data['z_difficulty'] = stats.zscore(input_data['difficulty'])

Add the ration of all correct and incorrect resposnes as a new feature: 

In [ ]:
input_data['correct_rate'] = np.sum(input_data['11898_corr'], input_data['37570_corr'], input_data['5968_corr'], 
                                   input_data['5969_corr'], input_data['6921_corr']) / np.sum(input_data['11898_incorr'], input_data['37570_incorr'], 
                                                                                            input_data['5968_incorr'],  input_data['6921_incorr'], 
                                                                                            input_data['5969_incorr'])

# 8. Run LogisticRegression and DecisionTreeClassifier on the data set

In [ ]:
all_features = input_data.columns.tolist()
print all_features

Remove _correct_ from features sets

In [ ]:
label = 'correct'
all_features.remove(label)
all_features.remove('user_id')
predictors = all_features
categorical_features = ['skill_id']

In [ ]:
model = LogisticRegression()
classification_model(model, input_data, predictors, label, categorical_features, 5)

In [ ]:
model = DecisionTreeClassifier()
classification_model(model, input_data, predictors, label, categorical_features, 5)